In [1]:
import os
import glob
import requests
from loguru import logger
from typing import List, Dict
from hcmus.core import pconf

2025-03-08 11:32:13.522 | INFO     | hcmus.core.pconf:<module>:5 - Load DotEnv: True


In [2]:
def chunk_list(lst, chunk_size):
    result = []
    for i in range(0, len(lst), chunk_size):
        result.append(lst[i:i + chunk_size])
    return result

In [12]:
def list_all_images(folder_path: str) -> List[str]: 
    image_files = []
    extensions = ["jpg", "png", "JPG", "PNG"]
    for ext in extensions:
        image_files.extend(glob.glob(f"{folder_path}/**/*.{ext}", recursive=True))

    return image_files

In [13]:
def get_filename_from_path(path: str) -> str:
    return os.path.basename(path)

In [14]:
def get_label_studio_headers() -> Dict[str, str]:
    headers = {
        "Authorization": f"Token {pconf.LABEL_STUDIO_API_KEY}"
    }
    return headers

In [36]:
# def get_uploaded_tasks():
#     headers = get_label_studio_headers()
#     endpoint = f"{pconf.LABEL_STUDIO_URL}/api/projects/{pconf.LABEL_STUDIO_PROJECT_ID}/tasks/"
#     response = requests.get(
#         endpoint,
#         headers=headers,
#     )
#     if response.status_code == 200:
#         tasks = response.json()
#         files = {task["data"].get("image", "") for task in tasks}  # Extract uploaded file paths
#         files = {get_filename_from_path(x) for x in files}
#         return files
#     else:
#         print("Error fetching tasks:", response.text)
#         return set()
def get_uploaded_tasks():
    """Fetch all tasks from Label Studio with pagination"""
    uploaded_files = set()
    page = 1
    page_size = 100  # You can increase this if needed
    headers = get_label_studio_headers()
    endpoint = f"{pconf.LABEL_STUDIO_URL}/api/projects/{pconf.LABEL_STUDIO_PROJECT_ID}/tasks/"

    while True:
        response = requests.get(
            endpoint,
            headers=headers,
            params={"page": page, "page_size": page_size},
        )

        if response.status_code != 200:
            logger.warning("Error fetching tasks:" + response.text)
            break

        tasks = response.json()
        if not tasks:  # Stop when no more tasks are returned
            break

        uploaded_files.update(task["data"].get("image", "") for task in tasks)
        page += 1  # Move to the next page

    return uploaded_files

In [37]:
def upload_file(file_path: str, uploaded_tasks):
    filename = get_filename_from_path(file_path)
    for task in uploaded_tasks:
        if filename not in task: continue
        logger.info(f"File already exists: {filename}") 
        return

    headers = get_label_studio_headers()
    with open(file_path, "rb") as f:
        files = {"file": f}
        endpoint = f"{pconf.LABEL_STUDIO_URL}/api/projects/{pconf.LABEL_STUDIO_PROJECT_ID}/import"
        response = requests.post(
            endpoint,
            files=files,
            headers=headers,
        )
    
    if response.status_code == 201:
        print(f"Uploaded: {file_path}")
        return response.json()
    else:
        print(f"Error uploading {file_path}: {response.text}")
        return None

def upload_multiple_files(chunk: List[str], uploaded_tasks):
    for file_path in chunk:
        upload_file(file_path, uploaded_tasks)

In [38]:
files = list_all_images(pconf.IMPORT_DATA_DIR) 
chunks = chunk_list(files, 50)
logger.info(f"Number of chunks: {len(chunks)}")

2025-03-08 11:42:54.482 | INFO     | __main__:<module>:3 - Number of chunks: 32


In [39]:
uploaded_tasks =  get_uploaded_tasks()
logger.info(f"Number of uploaded tasks: {len(uploaded_tasks)}")

2025-03-08 11:42:56.391 | WARNING  | __main__:get_uploaded_tasks:32 - Error fetching tasks:{"id":"5634c4e5-72ab-410c-a328-844105224cdd","status_code":404,"version":"1.16.0","detail":"Not found.","exc_info":null}
2025-03-08 11:42:56.393 | INFO     | __main__:<module>:2 - Number of uploaded tasks: 189


In [40]:
for chunk in chunks:
    upload_multiple_files(chunk, uploaded_tasks)

2025-03-08 11:42:59.342 | INFO     | __main__:upload_file:5 - File already exists: 20250303_122532.jpg
2025-03-08 11:42:59.344 | INFO     | __main__:upload_file:5 - File already exists: IMG_20250224_131820.jpg
2025-03-08 11:42:59.344 | INFO     | __main__:upload_file:5 - File already exists: 20250303_122533.jpg
2025-03-08 11:42:59.346 | INFO     | __main__:upload_file:5 - File already exists: IMG_20250224_131944.jpg
2025-03-08 11:42:59.346 | INFO     | __main__:upload_file:5 - File already exists: IMG_20250224_131945.jpg
2025-03-08 11:42:59.347 | INFO     | __main__:upload_file:5 - File already exists: IMG_20250224_131951.jpg
2025-03-08 11:42:59.347 | INFO     | __main__:upload_file:5 - File already exists: IMG_20250224_131947.jpg
2025-03-08 11:42:59.347 | INFO     | __main__:upload_file:5 - File already exists: 20250303_122535.jpg
2025-03-08 11:42:59.347 | INFO     | __main__:upload_file:5 - File already exists: IMG_20250224_131935.jpg
2025-03-08 11:42:59.348 | INFO     | __main__:upl

Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Juice/IMG_20250303_123358_428.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Juice/IMG_20250303_123419_655.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Juice/z6354735949699_11161808441deaa12fae74e3da93e146.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Juice/IMG_20250303_123253_613.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Juice/IMG_20250303_123245_319.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Juice/z6354735949715_976e3f26d07b9a68ea778941e1de69cb.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Juice/z6354735978681_325ce3c3fe38b5fcac78c637a453816b.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Juice/z6354735965264_7e9047ce8bfc940ec341f1dd2e61c3a0.jpg
Uploaded: /Volumes/Cucumber/Proj

2025-03-08 11:43:22.902 | INFO     | __main__:upload_file:5 - File already exists: 20250224_131108.jpg
2025-03-08 11:43:22.902 | INFO     | __main__:upload_file:5 - File already exists: 20250224_130558.jpg
2025-03-08 11:43:22.903 | INFO     | __main__:upload_file:5 - File already exists: 20250224_131733.jpg
2025-03-08 11:43:22.904 | INFO     | __main__:upload_file:5 - File already exists: 20250224_131055.jpg
2025-03-08 11:43:22.904 | INFO     | __main__:upload_file:5 - File already exists: 20250224_132004.jpg
2025-03-08 11:43:22.905 | INFO     | __main__:upload_file:5 - File already exists: z6354730397036_295654736e94695f8316d816004ae993.jpg
2025-03-08 11:43:22.906 | INFO     | __main__:upload_file:5 - File already exists: z6354730437417_88c4156f0a56b70c80cbb6c99e7e6a9f.jpg
2025-03-08 11:43:22.906 | INFO     | __main__:upload_file:5 - File already exists: 20250224_130758.jpg
2025-03-08 11:43:22.907 | INFO     | __main__:upload_file:5 - File already exists: 20250224_131042.jpg
2025-03-0

Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Sauces & Herbs & Spices/20250224_123105.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Home and kitchen - Cleaning supplies/IMG_20250224_125328.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Home and kitchen - Cleaning supplies/IMG_20250224_125314.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Home and kitchen - Cleaning supplies/IMG_20250224_123942.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Home and kitchen - Cleaning supplies/IMG_20250224_123229.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Home and kitchen - Cleaning supplies/IMG_20250224_124546.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Home and kitchen - Cleaning supplies/IMG_20250224_124552.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Home and kitchen - Cleaning supplies/IMG_20250224_124426.jpg
Uploaded: /Volumes

2025-03-08 11:49:04.993 | INFO     | __main__:upload_file:5 - File already exists: 20250303_122630.jpg


...

Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Instant Noodles/20250303_122843.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Instant Noodles/20250224_125610.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Instant Noodles/20250224_125210.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Instant Noodles/20250303_122847.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Instant Noodles/20250224_125358.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Instant Noodles/20250224_124734.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Instant Noodles/20250224_124906.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Instant Noodles/20250224_125629.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Instant Noodles/20250224_124858.jpg
Uploaded: /Volumes/Cucumber/Projects/item-iden

2025-03-08 11:50:34.699 | INFO     | __main__:upload_file:5 - File already exists: 20250303_123423.jpg


Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Beers, cider and Wine/IMG_20250303_124453_293.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Beers, cider and Wine/z6354730721848_b6d5096845762bb0f82d48fc0510e298.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Beers, cider and Wine/z6354730631399_9a584710c1f256dd12eeef18246996c2.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Beers, cider and Wine/z6354730784316_869e02cb9d3029590a144d59b10c2f16 (1).jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Beers, cider and Wine/IMG_20250303_124529_357.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Beers, cider and Wine/z6354730764751_2e0ae1ae948a9acc8e11a2e8cbc834e9 (1).jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Beers, cider and Wine/IMG_20250303_124137_378.jpg
Uploaded: /Volumes/Cu

2025-03-08 11:50:39.505 | INFO     | __main__:upload_file:5 - File already exists: 20250303_122727.jpg
2025-03-08 11:50:39.505 | INFO     | __main__:upload_file:5 - File already exists: 20250303_123411.jpg
2025-03-08 11:50:39.506 | INFO     | __main__:upload_file:5 - File already exists: IMG_20250224_132047.jpg
2025-03-08 11:50:39.506 | INFO     | __main__:upload_file:5 - File already exists: IMG_20250224_132053.jpg
2025-03-08 11:50:39.507 | INFO     | __main__:upload_file:5 - File already exists: 20250303_123404.jpg
2025-03-08 11:50:39.507 | INFO     | __main__:upload_file:5 - File already exists: 20250303_123438.jpg
2025-03-08 11:50:39.508 | INFO     | __main__:upload_file:5 - File already exists: IMG_20250224_132051.jpg
2025-03-08 11:50:39.509 | INFO     | __main__:upload_file:5 - File already exists: IMG_20250224_132044.jpg
2025-03-08 11:50:39.509 | INFO     | __main__:upload_file:5 - File already exists: 20250303_123413.jpg
2025-03-08 11:50:39.510 | INFO     | __main__:upload_file

Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Beverages - Beers, cider and Wine/IMG_20250303_124322_069.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Milk/IMG_20250303_124744_941.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Milk/IMG_20250303_124919_803.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Milk/IMG_20250303_124915_429.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Milk/IMG_20250303_122505_244.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Milk/IMG_20250303_124800_269.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Milk/z6354739761525_8fee840051bb368f306c74cb76428301.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Milk/z6354736007759_c5b9f2b9570c308cb71a30a15bcd62fb.jpg
Uploaded: /Volumes/Cucumber/Projects/item-identification/local/Food - Milk/IMG_20250303_125003_135.jpg
Upl